# Least Squares GAN (lsGAN)


Ref. 
MAO, Xudong et al.   
Least squares generative adversarial networks.   
In: Proceedings of the IEEE International Conference on Computer Vision. 2017. p. 2794-2802.

![Comparing GAN with lsGAN](equations.png)

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam

Using TensorFlow backend.


## generator

In [3]:
def build_generator(img_shape, latent_dim):

    model = Sequential()

    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    noise = Input(shape=(latent_dim,))
    img = model(noise)

    return Model(noise, img)

## discriminator

In [4]:
def build_discriminator(img_shape):

    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    # (!!!) No softmax
    model.add(Dense(1))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

## helper

In [5]:
def sample_images(G, latent_dim, epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = G.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

In [6]:
# train

In [7]:
def train(G, D, combined,
          latent_dim, channels,
          epochs, batch_size=128, sample_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random batch of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        # Sample noise as generator input
        noise = np.random.normal(0, 1, (batch_size, latent_dim))

        # Generate a batch of new images
        gen_imgs = G.predict(noise)

        # Train the discriminator
        d_loss_real = D.train_on_batch(imgs, valid)
        d_loss_fake = D.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


        # ---------------------
        #  Train Generator
        # ---------------------

        g_loss = combined.train_on_batch(noise, valid)

        # Plot the progress
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        # If at save interval => save generated image samples
        if epoch % sample_interval == 0 or epoch == epochs - 1:
            sample_images(G, latent_dim, epoch)

## main()

In [8]:
for d in ['images', 'save-models']:
    if not os.path.exists(d):
        os.makedirs(d)

In [9]:
# initialization
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

In [10]:
# create optimizer
optimizer = Adam(0.0002, 0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [11]:
# Build the generator
G = build_generator(img_shape, latent_dim)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dense_2 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dense_3 (Dense)              (None, 1024)             

In [12]:
# Build and compile the discriminator
D = build_discriminator(img_shape)
D.compile(loss='mse',
          optimizer=optimizer,
          metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
________________________________________________

In [13]:
# The generator takes noise as input and generated imgs
z = Input(shape=(latent_dim,))
img = G(z)

In [14]:
# For the combined model we will only train the generator
D.trainable = False

In [15]:
# The valid takes generated images as input and determines validity
valid = D(img)

In [16]:
# The combined model  (stacked generator and discriminator)
# Trains generator to fool discriminator
combined = Model(z, valid)
# (!!!) Optimize w.r.t. MSE loss instead of crossentropy
combined.compile(loss='mse', optimizer=optimizer)

In [17]:
# epochs=30000
epochs=6000

In [18]:
train(G, D, combined,
      latent_dim, channels,
      epochs=epochs, batch_size=32, sample_interval=200)

c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.322045, acc.: 67.19%] [G loss: 0.550221]


c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 2.035805, acc.: 50.00%] [G loss: 0.639625]
2 [D loss: 0.150175, acc.: 78.12%] [G loss: 0.585763]
3 [D loss: 0.257448, acc.: 67.19%] [G loss: 0.509979]
4 [D loss: 0.168925, acc.: 75.00%] [G loss: 0.565450]
5 [D loss: 0.138653, acc.: 81.25%] [G loss: 0.566457]
6 [D loss: 0.124201, acc.: 87.50%] [G loss: 0.658915]
7 [D loss: 0.117655, acc.: 79.69%] [G loss: 0.661321]
8 [D loss: 0.127982, acc.: 85.94%] [G loss: 0.822388]
9 [D loss: 0.114002, acc.: 81.25%] [G loss: 0.916730]
10 [D loss: 0.178487, acc.: 75.00%] [G loss: 1.112351]
11 [D loss: 0.190927, acc.: 75.00%] [G loss: 0.941258]
12 [D loss: 0.207223, acc.: 67.19%] [G loss: 1.204484]
13 [D loss: 0.206764, acc.: 73.44%] [G loss: 1.055375]
14 [D loss: 0.388627, acc.: 43.75%] [G loss: 1.199451]
15 [D loss: 0.595122, acc.: 43.75%] [G loss: 0.880832]
16 [D loss: 0.748525, acc.: 37.50%] [G loss: 1.051697]
17 [D loss: 0.367882, acc.: 46.88%] [G loss: 1.074713]
18 [D loss: 0.157338, acc.: 84.38%] [G loss: 1.284896]
19 [D loss: 0.11547

152 [D loss: 0.037300, acc.: 100.00%] [G loss: 1.094946]
153 [D loss: 0.074329, acc.: 93.75%] [G loss: 0.969122]
154 [D loss: 0.052065, acc.: 100.00%] [G loss: 1.149478]
155 [D loss: 0.061543, acc.: 98.44%] [G loss: 0.881486]
156 [D loss: 0.051679, acc.: 100.00%] [G loss: 1.119415]
157 [D loss: 0.039520, acc.: 100.00%] [G loss: 0.923723]
158 [D loss: 0.046082, acc.: 98.44%] [G loss: 1.131049]
159 [D loss: 0.064027, acc.: 96.88%] [G loss: 1.111001]
160 [D loss: 0.054797, acc.: 95.31%] [G loss: 1.000921]
161 [D loss: 0.043787, acc.: 100.00%] [G loss: 0.909855]
162 [D loss: 0.053215, acc.: 96.88%] [G loss: 1.114789]
163 [D loss: 0.071854, acc.: 96.88%] [G loss: 0.990222]
164 [D loss: 0.074738, acc.: 98.44%] [G loss: 1.001971]
165 [D loss: 0.127306, acc.: 92.19%] [G loss: 0.849983]
166 [D loss: 0.071274, acc.: 96.88%] [G loss: 1.031576]
167 [D loss: 0.071458, acc.: 92.19%] [G loss: 1.057764]
168 [D loss: 0.078678, acc.: 89.06%] [G loss: 1.129210]
169 [D loss: 0.097262, acc.: 90.62%] [G los

...

5386 [D loss: 0.216902, acc.: 75.00%] [G loss: 0.538653]
5387 [D loss: 0.190008, acc.: 75.00%] [G loss: 0.525725]
5388 [D loss: 0.238868, acc.: 65.62%] [G loss: 0.579798]
5389 [D loss: 0.181320, acc.: 68.75%] [G loss: 0.585040]
5390 [D loss: 0.204023, acc.: 68.75%] [G loss: 0.533190]
5391 [D loss: 0.257031, acc.: 57.81%] [G loss: 0.500574]
5392 [D loss: 0.179650, acc.: 71.88%] [G loss: 0.558873]
5393 [D loss: 0.181738, acc.: 73.44%] [G loss: 0.573589]
5394 [D loss: 0.156526, acc.: 78.12%] [G loss: 0.612513]
5395 [D loss: 0.187946, acc.: 71.88%] [G loss: 0.607549]
5396 [D loss: 0.216131, acc.: 70.31%] [G loss: 0.544138]
5397 [D loss: 0.163418, acc.: 75.00%] [G loss: 0.657543]
5398 [D loss: 0.150851, acc.: 79.69%] [G loss: 0.604990]
5399 [D loss: 0.214663, acc.: 68.75%] [G loss: 0.544424]
5400 [D loss: 0.194881, acc.: 70.31%] [G loss: 0.548205]
5401 [D loss: 0.229507, acc.: 65.62%] [G loss: 0.502680]
5402 [D loss: 0.182829, acc.: 68.75%] [G loss: 0.558068]
5403 [D loss: 0.199062, acc.: 7

5530 [D loss: 0.157695, acc.: 75.00%] [G loss: 0.577440]
5531 [D loss: 0.148512, acc.: 79.69%] [G loss: 0.522033]
5532 [D loss: 0.221048, acc.: 68.75%] [G loss: 0.527372]
5533 [D loss: 0.190258, acc.: 70.31%] [G loss: 0.548580]
5534 [D loss: 0.210287, acc.: 67.19%] [G loss: 0.555962]
5535 [D loss: 0.165310, acc.: 75.00%] [G loss: 0.602183]
5536 [D loss: 0.161704, acc.: 75.00%] [G loss: 0.625645]
5537 [D loss: 0.193931, acc.: 65.62%] [G loss: 0.542803]
5538 [D loss: 0.212267, acc.: 70.31%] [G loss: 0.525621]
5539 [D loss: 0.213192, acc.: 67.19%] [G loss: 0.509775]
5540 [D loss: 0.195841, acc.: 67.19%] [G loss: 0.602382]
5541 [D loss: 0.199327, acc.: 71.88%] [G loss: 0.557133]
5542 [D loss: 0.231243, acc.: 56.25%] [G loss: 0.470192]
5543 [D loss: 0.158126, acc.: 78.12%] [G loss: 0.524142]
5544 [D loss: 0.218933, acc.: 70.31%] [G loss: 0.576968]
5545 [D loss: 0.224839, acc.: 65.62%] [G loss: 0.529572]
5546 [D loss: 0.181813, acc.: 78.12%] [G loss: 0.646807]
5547 [D loss: 0.154083, acc.: 8

5674 [D loss: 0.174553, acc.: 78.12%] [G loss: 0.616095]
5675 [D loss: 0.211777, acc.: 67.19%] [G loss: 0.549246]
5676 [D loss: 0.194900, acc.: 68.75%] [G loss: 0.558059]
5677 [D loss: 0.163265, acc.: 82.81%] [G loss: 0.579127]
5678 [D loss: 0.165364, acc.: 71.88%] [G loss: 0.581411]
5679 [D loss: 0.188439, acc.: 73.44%] [G loss: 0.643459]
5680 [D loss: 0.207761, acc.: 62.50%] [G loss: 0.534111]
5681 [D loss: 0.175514, acc.: 68.75%] [G loss: 0.610455]
5682 [D loss: 0.180266, acc.: 76.56%] [G loss: 0.565161]
5683 [D loss: 0.209742, acc.: 65.62%] [G loss: 0.602358]
5684 [D loss: 0.173000, acc.: 75.00%] [G loss: 0.609376]
5685 [D loss: 0.240140, acc.: 60.94%] [G loss: 0.572326]
5686 [D loss: 0.186129, acc.: 75.00%] [G loss: 0.540825]
5687 [D loss: 0.193939, acc.: 70.31%] [G loss: 0.525370]
5688 [D loss: 0.166928, acc.: 73.44%] [G loss: 0.621456]
5689 [D loss: 0.209415, acc.: 64.06%] [G loss: 0.568326]
5690 [D loss: 0.192838, acc.: 64.06%] [G loss: 0.559197]
5691 [D loss: 0.211943, acc.: 7

5818 [D loss: 0.186638, acc.: 78.12%] [G loss: 0.635606]
5819 [D loss: 0.192745, acc.: 65.62%] [G loss: 0.545890]
5820 [D loss: 0.182486, acc.: 71.88%] [G loss: 0.596057]
5821 [D loss: 0.214922, acc.: 62.50%] [G loss: 0.548777]
5822 [D loss: 0.161243, acc.: 76.56%] [G loss: 0.596071]
5823 [D loss: 0.232826, acc.: 60.94%] [G loss: 0.573432]
5824 [D loss: 0.184224, acc.: 73.44%] [G loss: 0.559002]
5825 [D loss: 0.215260, acc.: 68.75%] [G loss: 0.545952]
5826 [D loss: 0.171977, acc.: 75.00%] [G loss: 0.568501]
5827 [D loss: 0.192632, acc.: 70.31%] [G loss: 0.552051]
5828 [D loss: 0.214029, acc.: 71.88%] [G loss: 0.539003]
5829 [D loss: 0.204455, acc.: 73.44%] [G loss: 0.514431]
5830 [D loss: 0.211797, acc.: 70.31%] [G loss: 0.583893]
5831 [D loss: 0.190436, acc.: 75.00%] [G loss: 0.548058]
5832 [D loss: 0.163798, acc.: 79.69%] [G loss: 0.654367]
5833 [D loss: 0.157549, acc.: 73.44%] [G loss: 0.652349]
5834 [D loss: 0.188102, acc.: 70.31%] [G loss: 0.544743]
5835 [D loss: 0.246203, acc.: 6

5963 [D loss: 0.190555, acc.: 71.88%] [G loss: 0.571098]
5964 [D loss: 0.188917, acc.: 71.88%] [G loss: 0.593225]
5965 [D loss: 0.196837, acc.: 75.00%] [G loss: 0.603752]
5966 [D loss: 0.169213, acc.: 73.44%] [G loss: 0.615991]
5967 [D loss: 0.191643, acc.: 70.31%] [G loss: 0.576759]
5968 [D loss: 0.181140, acc.: 71.88%] [G loss: 0.638367]
5969 [D loss: 0.206416, acc.: 70.31%] [G loss: 0.575354]
5970 [D loss: 0.234626, acc.: 51.56%] [G loss: 0.529649]
5971 [D loss: 0.158058, acc.: 75.00%] [G loss: 0.657905]
5972 [D loss: 0.155914, acc.: 78.12%] [G loss: 0.577565]
5973 [D loss: 0.229021, acc.: 57.81%] [G loss: 0.591568]
5974 [D loss: 0.227403, acc.: 60.94%] [G loss: 0.628639]
5975 [D loss: 0.184872, acc.: 70.31%] [G loss: 0.600756]
5976 [D loss: 0.189665, acc.: 70.31%] [G loss: 0.649689]
5977 [D loss: 0.190044, acc.: 71.88%] [G loss: 0.558875]
5978 [D loss: 0.205808, acc.: 65.62%] [G loss: 0.570496]
5979 [D loss: 0.231454, acc.: 60.94%] [G loss: 0.551742]
5980 [D loss: 0.189934, acc.: 6